In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
period = 'PM'

In [3]:
if period == 'AM':
    hr_list = [7,8,9,10,11]
    departure_hour = [7]
elif period == 'MD':
    hr_list = [11,12,13,14,15]
    departure_hour = [11]
elif period == 'PM':
    hr_list = [17,18,19,20,21]
    departure_hour = [17]
elif period == 'EV':
    hr_list = [20,21,22,23,0]
    departure_hour = [20]
elif period == 'EM':
    hr_list = [4,5,6,7,8]
    departure_hour = [4]

In [4]:
stop_times_reduced = pd.read_csv('gtfs_2019-11-14/stop_times_reduced.csv')

In [5]:
#stop_times_reduced = stop_times_reduced.drop(columns = 'stop_sequence').rename(columns = {'stop_sequence_new':'stop_sequence'})

In [6]:
stop_times_reduced = stop_times_reduced[stop_times_reduced['hr'].isin(hr_list)]

stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])

stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [7]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new', 'route_short_name'],
                           right_on = ['trip_id', 'stop_sequence_new','route_short_name'], suffixes = ['_o', '_d'])

In [8]:
stop_times_od = stop_times_od.drop(columns = ['dist_o', 'dist_d', 'stop_id_o', 'stop_id_d'])

In [9]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [10]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)

In [11]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [12]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')



In [13]:
stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

In [14]:
stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

In [15]:
stop_times_od['node_o'] = stop_times_od['INT_ID_o'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_o'].astype(
    str) + '-' + stop_times_od['min_o'].astype(int).astype(str)
stop_times_od['node_d'] = stop_times_od['INT_ID_d'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_d'].astype(
    str) + '-' + stop_times_od['min_d'].astype(int).astype(str)

In [16]:
trips_weekday = pd.read_csv('gtfs_2019-11-14/trips_weekday.csv')
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'direction_id']], how = 'inner')

In [17]:
stop_times_od['link'] = '0' + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-' + stop_times_od['trip_id'].astype(str) + '-' + stop_times_od['direction_id'].astype(str)+ '_' + stop_times_od['node_o'] + '_' + stop_times_od['node_d']

In [18]:
stop_times_od['cost'] = stop_times_od['cost'].astype(int)

In [19]:
walk_links = pd.read_csv('int_tts_walk_time19.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000

# Graph Build

In [20]:
G = nx.DiGraph()

## Node Times

In [21]:
nodes = list((stop_times_od['INT_ID_o'].append(stop_times_od['INT_ID_d'])).drop_duplicates())

In [22]:
node_list = []
for node in nodes:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(node)) + '-0-' + str(hr) + '-' + str(minute)
            node_list.append(node_id)

In [23]:
G.add_nodes_from(node_list)

## Boarding Nodes

In [24]:
boarding_nodes = list(stop_times_od['node_o'].drop_duplicates())
G.add_nodes_from(boarding_nodes)

## Alighting Nodes

In [25]:
alighting_nodes = list(stop_times_od['node_d'].drop_duplicates())
G.add_nodes_from(alighting_nodes)

## Travel Links

In [26]:
link_list = []
for index, row in stop_times_od.iterrows():
    attributes = {'cost': row['cost']}
    link = (row['node_o'], row['node_d'], attributes)
    link_list.append(link)
    
G.add_edges_from(link_list)

## Waiting Links

In [27]:
transfer_list = []
for node in nodes:
    next_hr = hr_list[0]
    next_minute = 0
    node = int(node)
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            if current_minute == 59:
                next_minute = 0
                next_hr = current_hr + 1
                
            else:
                next_minute = current_minute + 1
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '1-0-0-9_' + node_o + '_' + node_d
            
            attributes = {'cost': 1}
            
            
            link = (node_o, node_d, attributes)
            transfer_list.append(link)
    

In [28]:
G.add_edges_from(transfer_list)

## Boarding Links

In [29]:
boarding_link = []
for node in boarding_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    boarding_hr = int(node_split[2])
    boarding_minute = int(node_split[3])

    stop_minute = boarding_minute - 2
    if stop_minute < 0:
        stop_hr = boarding_hr -1
        stop_minute = 60 + stop_minute
    else:
        stop_hr = boarding_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (stop_node, node, attributes)
    
    boarding_link.append(link)

In [30]:
G.add_edges_from(boarding_link)

## Alighting Links

In [31]:
alighting_link = []
for node in alighting_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    alighting_hr = int(node_split[2])
    alighting_minute = int(node_split[3])

    stop_minute = alighting_minute + 2
    if stop_minute > 59:
        stop_hr = alighting_hr + 1
        stop_minute = stop_minute - 60
    else:
        stop_hr = alighting_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (node, stop_node, attributes)
    
    alighting_link.append(link)

In [32]:
G.add_edges_from(alighting_link)

## TTS Zone Origins

In [33]:
tts_zones = list(walk_links['gta06'].drop_duplicates())

node_stationary_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-0-' + str(hr) + '-' + str(minute)
            node_stationary_list.append(node_id)
            
G.add_nodes_from(node_stationary_list)

## TTS Zone Destinations

In [34]:
zone_destinations = []
for zone in tts_zones:
    node_id = str(int(zone)) + '-0-99-99'
    zone_destinations.append(node_id)

In [35]:
G.add_nodes_from(zone_destinations)

## From TTS to Intersections

In [36]:
walk_link_outbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in departure_hour:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(zone) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-0_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_outbound.append(link)

## From Intersections To TTS

In [37]:
walk_link_inbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
                
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(zone) + '-0-99-99'
            
            link_name = '2-0-0-1_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_inbound.append(link)

In [38]:
G.add_edges_from(walk_link_outbound)

In [39]:
G.add_edges_from(walk_link_inbound)

In [40]:
nx.write_graphml(G,'networks/scheduled/2019-11-14_' + period + '.graphml')
stop_times_od.to_csv('networks/scheduled/2019-11-14_' + period + '.csv', index = False)

In [41]:
list(G.neighbors('151-0-17-5'))

['151-0-17-6',
 '151-504-17-7',
 '151-508-17-7',
 '1015-0-99-99',
 '1016-0-99-99',
 '1024-0-99-99',
 '1025-0-99-99',
 '1026-0-99-99',
 '1028-0-99-99',
 '1030-0-99-99',
 '1031-0-99-99',
 '1032-0-99-99',
 '1033-0-99-99',
 '1034-0-99-99',
 '1035-0-99-99',
 '1036-0-99-99',
 '1037-0-99-99',
 '1038-0-99-99',
 '1040-0-99-99',
 '1048-0-99-99',
 '1049-0-99-99',
 '1050-0-99-99',
 '1051-0-99-99',
 '1052-0-99-99',
 '1053-0-99-99',
 '1054-0-99-99',
 '1055-0-99-99',
 '1056-0-99-99',
 '1057-0-99-99',
 '1058-0-99-99',
 '1059-0-99-99',
 '1060-0-99-99',
 '1061-0-99-99',
 '1062-0-99-99',
 '1063-0-99-99',
 '1064-0-99-99',
 '1065-0-99-99',
 '1066-0-99-99',
 '1067-0-99-99',
 '1068-0-99-99',
 '1069-0-99-99',
 '1076-0-99-99',
 '1077-0-99-99',
 '1078-0-99-99',
 '1079-0-99-99',
 '1080-0-99-99',
 '1090-0-99-99',
 '1081-0-99-99']